In [11]:
import numpy as np
import pandas as pd
from math import log2

def entropy(data):
    classes, counts = np.unique(data, return_counts=True)
    total = np.sum(counts)
    return -sum((count / total) * log2(count / total) for count in counts)

def information_gain(data, feature, target):
    total_entropy = entropy(target)
    values, counts = np.unique(data[feature], return_counts=True)
    weighted_entropy = sum((counts[i] / np.sum(counts)) * entropy(target[data[feature] == values[i]]) for i in range(len(values)))
    return total_entropy - weighted_entropy

def id3(data, features, target, parent_value=None):
    if len(np.unique(target)) == 1:
        return np.unique(target)[0]
    if len(data) == 0 or len(features) == 0:
        return np.unique(parent_value, return_counts=True)[0][0]
    parent_value = target
    gains = {feature: information_gain(data, feature, target) for feature in features}
    best_feature = max(gains, key=gains.get)
    tree = {best_feature: {}}
    for value in np.unique(data[best_feature]):
        subset = data[data[best_feature] == value]
        subtree = id3(subset, [f for f in features if f != best_feature], subset[target.name])
        tree[best_feature][value] = subtree
    return tree

data = pd.DataFrame({
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Rainy', 'Overcast', 'Sunny', 'Sunny', 'Rainy', 'Sunny', 'Overcast', 'Overcast', 'Rainy'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
    'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
})

features = list(data.columns[:-1])
target = data['Play']
tree = id3(data, features, target)

print("Decision Tree:")
print(tree)


Decision Tree:
{'Outlook': {'Overcast': 'Yes', 'Rainy': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
